In [1]:
import networkx as nx
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import random

In [2]:
from auxiliaries import *

random.seed(20)

In [47]:
# types of routes
types = {
    0: "tram",
    1: "subway",
    2: "rail",
    3: "bus",
    4: "ferry",
    5: "walking",
}

In [48]:
# read in public transport network of berlin
berlin, berlin_nodes = read_in_network("berlin", "combined")

# read in walking network
berlin_walking, nodes_t = read_in_network("berlin", "walk")
# rename d_walk column to duration_avg
berlin_walking = berlin_walking.rename(columns={"d_walk": "duration_avg"})
# add n_vehicles and route_I_counts columns, set both to 0
berlin_walking["n_vehicles"] = 0
berlin_walking["route_I_counts"] = 0
# add route_type column, set to 5 (walking)
berlin_walking["route_type"] = 5

# add berlin_walking to berlin
berlin_full = pd.concat([berlin, berlin_walking], ignore_index=True)

In [49]:
# count route_type counts in berlin_full
berlin_full["route_type"].value_counts()

route_type
5    18125
3    10208
0      990
2      507
1      366
4        8
Name: count, dtype: int64

In [50]:
# drop all routes with route_type 4
berlin_full = berlin_full[berlin_full["route_type"] != 4]

In [51]:
berlin_full["route_type"].value_counts()

route_type
5    18125
3    10208
0      990
2      507
1      366
Name: count, dtype: int64

In [52]:
# convert to graph
G = convert_to_graph(berlin_full)
pos = add_positions(G, berlin_nodes)

In [53]:
G.get_edge_data(10924, 10920)

{0: {'duration_avg': 120.0, 'route_type': 3},
 1: {'duration_avg': 873.0, 'route_type': 5}}

In [54]:
len(G.edges())

30196

## Connected component

The Berlin network is not a connected component, but as we see below, the largest connected component is very large (only 8 nodes are not connected to it). Therefore we can just drop those few nodes and work only with the largest connected component.

In [55]:
min(nx.connected_components(G), key=len)

{10661, 10662}

In [56]:
# drop nodes 10661 and 10662
G.remove_nodes_from([10661, 10662])

In [57]:
# length of each connected component
for component in nx.connected_components(G):
    print(len(component))

4598


## Functions for experiments

In [58]:
def travel_time(a, b):
    """Rough estimate of travel time between two nodes."""
    return nx.shortest_path_length(G, a, b, weight="duration_avg")

In [61]:
nx.shortest_path_length(G, 10924, 345, weight="duration_avg")

1776.245733788396

In [63]:
def average_travel_time(G):
    """Average travel time between all pairs of nodes in G."""
    return nx.average_shortest_path_length(G, weight="duration_avg")


def full_average_travel_time(G):
    """
    calculate average travel time for each component of G
    and weight it by the number of nodes in the component
    """
    components = nx.connected_components(G)
    total = 0
    for component in components:
        total += len(component) * average_travel_time(G.subgraph(component))
    return total / len(G)


def random_sample(nodes, size):
    """
    Randomly sample a subset of nodes from the graph.
    """
    # take sample of nodes
    sample = random.sample(list(nodes), size)
    return sample


def sample_average_travel_time(sample, G):
    """
    Average travel time between all pairs of nodes in the sample.
    """
    # for each pair in sample, calculate shortest path length in G
    # and average over all pairs
    total = 0
    for a in sample:
        for b in sample:
            total += travel_time(a, b)
    return total / (len(sample) ** 2)

In [65]:
# only take largest connected component
sample_average_travel_time(random_sample(G.nodes, 100), G)

In [99]:
# function to perform random percolation on a graph
def random_percolation(G, p):
    """
    Random percolation on a graph.
    :param G: graph
    :param p: probability of edge removal
    """
    G_copy = G.copy()
    for edge in G_copy.edges:
        r = random.random()
        edgetype = G_copy.get_edge_data(edge[0], edge[1])["route_type"]
        if r < (p/10) and edgetype != 5:
            G_copy.remove_edge(edge[0], edge[1])
    return G_copy

In [182]:
G.get_edge_data()

{0: {'duration_avg': 120.0, 'route_type': 3},
 1: {'duration_avg': 120.0, 'route_type': 3}}

In [178]:
G.edges()

MultiEdgeDataView([(10924, 10920), (10924, 10794), (10924, 10794), (10924, 10435), (10924, 10435), (10924, 10436), (10924, 10436), (10924, 10928), (10920, 10799), (10794, 10447), (10794, 10443), (10435, 10800), (10435, 10800), (10435, 10425), (10435, 10425), (10435, 10798), (10436, 2208), (10436, 2208), (10925, 10492), (10925, 10492), (10925, 10471), (10925, 10471), (10492, 10926), (10492, 10926), (10492, 10927), (10492, 10487), (10471, 10429), (10471, 10429), (10471, 10482), (10471, 10483), (10471, 10483), (10471, 10469), (10471, 10469), (10471, 10486), (10471, 10486), (10471, 10487), (10471, 10487), (10471, 10489), (10471, 10489), (10471, 10493), (10471, 10902), (10471, 10902), (10471, 10933), (10926, 10490), (10926, 10490), (10490, 10487), (10490, 10487), (10490, 10473), (10490, 10473), (10927, 10491), (2736, 2738), (2736, 2738), (2736, 2735), (2736, 2737), (2736, 2737), (2736, 2768), (2738, 2737), (2738, 2754), (2738, 2754), (2738, 2739), (2738, 2739), (2738, 2739), (2738, 2739), (

In [117]:
G_perc = random_percolation(G, 100)

In [118]:
len(G_perc.edges())

5861